In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import datetime
from helpers import *
from implementations import *

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../../train.csv' # TODO: download train data and supply path here 
y_starting, tX_starting, ids = load_csv_data(DATA_TRAIN_PATH,sub_sample=False)

In [3]:
y=y_starting.copy()
tX=tX_starting.copy()

# Basic functions implemented

For the testing of the implementation of the 6 functions we did not preprocess the dataset.
The aim of this part is just to show that they work correctly

In [4]:
tX,tX_test,y,y_test=split_data(tX,y,0.5)
print(tX.shape)

[ 82155 219628 116163 ..., 115864 203703  15449]
(125000, 30)


### Least squares

In [5]:
w,loss=least_squares(y,tX)

In [6]:
evaluate(y,tX,w)

0.74474399999999996

The basic least square method achieve a 74% accuracy

### Ridge regression

In [ ]:
lamb=1

In [ ]:
w,loss=ridge_regression(y,tX,lamb)

In [ ]:
evaluate(y,tX,w)

The accuracy slightly decreases using the ridge regression. It was expected. The problem in the standard case is well posed, so the ridge regression is not convenient

### Gradient Descent

In [ ]:
initial_w=np.zeros(tX.shape[1])
gamma=0.0000001

In [ ]:
w,loss=least_squares_GD(y,tX,initial_w,10000,gamma)

In [ ]:
evaluate(y,tX,w)

Even with a lot of iterations the gradient descent is not able to match the results obtained with the least squares.


### Stochastic Gradient Descent

In [ ]:
initial_w=np.zeros(tX.shape[1])
gamma=0.00000001

In [ ]:
w,loss=least_squares_SGD(y,tX,initial_w,100000,gamma)

In [ ]:
evaluate(y,tX,w)

Stochastic Gradient Descent is not able to achieve a better result than least square.
The fastest iteration step time allows for more iterations than the standard GD.
It is however necessary a lower step size to keep the method stable.

### Logistic regression

In [ ]:
y_log=y.copy()
y_log[y_log==-1]=0
y_log

In [ ]:
initial_w=np.zeros(tX.shape[1])
gamma=0.00000000001

In [ ]:
w,loss=logistic_regression(y_log,tX,initial_w,1000,gamma)

In [ ]:
evaluate(y,tX,w)

### Regularized Logistic Regression

In [ ]:
y_log=y.copy()
y_log[y_log==-1]=0
y_log

In [ ]:
initial_w=np.zeros(tX.shape[1])
gamma=0.00000000001
lamb=1e-05

In [ ]:
w,loss=reg_logistic_regression(y_log,tX,lamb,initial_w,1000,gamma)

In [ ]:
evaluate(y,tX,w)

The results are quite similar to the standard logistic regression

### Regularized Logistic Regression Newton

In [ ]:
y_log=y.copy()
y_log[y_log==-1]=0
y_log

In [ ]:
initial_w=np.zeros(tX.shape[1])
gamma=0.00000000001
lamb=1e-05

In [ ]:
w,loss=reg_logistic_regression_newton(y_log,tX,lamb,initial_w,100,gamma)

In [ ]:
evaluate(y,tX,w)

The results improved over the standard logistic regression. 
Few iterations are necessary to achieve a good result compared to the previous methods.

## Generate predictions and save ouput in csv format for submission:

In [14]:
DATA_TEST_PATH = '../../test.csv' # TODO: download train data and supply path here 
_, tX_final_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [17]:
OUTPUT_PATH = '../../predictions_try.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_final)
#y_train,y_test = logistic_regression(y_train,tX_train,tX_final_test,0.48)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)
print(y_pred)

[-1. -1.  1. ..., -1. -1. -1.]
